In [1]:
import pandas as pd
from pythainlp import word_tokenize as tokenize
from pythainlp.util import *
from tqdm.auto import tqdm
import os, glob, sys
from collections import Counter
sys.path.append('../Cython')
from Method import *
tqdm.pandas()

C:\Users\Alps\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
abs_path = os.path.abspath(r'..\assets\SplitData')
# print(abs_path)
os.chdir(abs_path)

In [3]:
NEWMM = "newmm"
ATTACUT = "attacut"

def word_break(news,engine):
    segment = tokenize(news,keep_whitespace=False,engine=engine)
    return segment

In [4]:
def LCS(atta_list, newmm_list):
    l = len(atta_list)
    lcs_atta = set()
    lcs_newmm = set()
    for i in range(l-1):
        w1_atta = atta_list[i]
        len_w1_atta = len(w1_atta)
        w1_newmm = newmm_list[i]
        len_w1_newmm = len(w1_newmm)
        for j in range(i+1,l):
            w2_atta = atta_list[j]
            len_w2_atta = len(w2_atta)
            w2_newmm = newmm_list[j]
            len_w2_newmm = len(w2_newmm)
            lcs_atta_list,maxl_atta = [],0
            lcs_newmm_list,maxl_newmm = [],0
            if i != j:
                lcs_atta_list,maxl_atta = lcs(w1_atta,w2_atta,len_w1_atta,len_w2_atta)
                lcs_newmm_list,maxl_newmm = lcs(w1_newmm,w2_newmm,len_w1_newmm,len_w2_newmm)
                if lcs_atta_list != []:
                    lcs_atta.update(lcs_atta_list)
                if lcs_newmm_list != []:
                    lcs_newmm.update(lcs_newmm_list)

    return lcs_atta, lcs_newmm

In [5]:
def find_stop_keyword_list(month, threshold):   
    newmm_word_list = []
    attacut_word_list = []

    for file in tqdm(glob.glob(f"2021_{month:02d}_*")):
        df = pd.read_csv(file)
        list_column = df.columns.values.tolist()
        column_name = list_column[1]

        newmm_WB = []
        attacut_WB = []

        for news in df[column_name]:
            newmm_WB.append(word_break(str(news), NEWMM))
            attacut_WB.append(word_break(str(news), ATTACUT))
        
        attacut_word, newmm_word = LCS(attacut_WB, newmm_WB)
        
        newmm_word_list.extend(list(newmm_word))
        attacut_word_list.extend(list(attacut_word))

    newmm_counter = Counter(newmm_word_list)
    attacut_counter = Counter(attacut_word_list)

    result = {key:value for key,value in newmm_counter.items() if value >= threshold}
    stopwords = list(result.keys())

    textfile = open(f"../StopKeywordList/newmm_stopkeyword_m{month:02d}_th{threshold:02d}.txt", "w", encoding='utf-8')
    for element in stopwords:
        textfile.write(element + "\n")
    textfile.close()

    result = {key:value for key,value in attacut_counter.items() if value >= threshold}
    stopwords = list(result.keys())

    textfile = open(f"../StopKeywordList/attacut_stopkeyword_m{month:02d}_th{threshold:02d}.txt", "w", encoding='utf-8')
    for element in stopwords:
        textfile.write(element + "\n")
    textfile.close()

In [6]:
find_stop_keyword_list(10,20)

100%|██████████| 27/27 [34:27<00:00, 76.59s/it]
